In [1]:
using Surrogates,Flux,Statistics,DifferentialEquations,Plots,Suppressor,MLDataUtils

In [2]:
lb=Float32[0.0,5.0]
ub=Float32[30.0,10.0]

2-element Vector{Float32}:
 30.0
 10.0

In [3]:
#Define scaler functions to preprocess data
vecscaler(x)=(x.-lb)./(ub.-lb)
scaler(x)= ((x[1]- lb[1])/(ub[1]-lb[1]),(x[2]- lb[2])/(ub[2]-lb[2]))
invscaler(x_sc)=(x_sc[1]*(ub[1]-lb[1]) + lb[1],x_sc[2]*(ub[2]-lb[2]) + lb[2])
output_scaler(x)= x ./ 2
inv_output_scaler(x)=x.*2

inv_output_scaler (generic function with 1 method)

In [4]:
function DEX_Lpt(dose)
    return Lpt = 1.131e-06 - dose*(dose*(dose*(dose*((6.23e-13*dose) + 5.96e-11) - 5.61e-09) + 1.272e-07) - 8.797e-07)
end

function DEX_Kt(dose)
    return Kt = dose*(dose*(dose*(dose*((1.139e-11*dose) - 8.389e-10) + 2.183e-08) - 2.421e-07) + 1.093e-06) + 3.798e-07
end

function Diffusion(rs)
    Diam_parc = 2*rs
    return D = 1.981e-06*Diam_parc^(-1.157) + 2.221e-08
end

function Blood_half_life(rs)
    Diam_parc = 2*rs
    a1 = 1081
    b1 = -16.63
    c1 = 84.82
    a2 = 517.4
    b2 = 65.61
    c2 = 996.6
    return kd = (a1*exp(-((Diam_parc-b1)/c1)^2) + a2*exp(-((Diam_parc-b2)/c2)^2))*60
end
function Isolated_Pressure(N,R,Lpt,Svt,Kt,Pvv)

    # Pressure Profile
    att = R*sqrt(Lpt*Svt/Kt)

    r = range(0,step = R/(N-1),length = N)
    r = r/R
    dr = 1.0/(N-1)
    ivdr2 = 1.0./dr^2
    att2 = att^2
    A = zeros(typeof(Lpt),N,N)
    F = zeros(typeof(Lpt),N,1)

    M = N
    for i in 2:M-1
        A[i,i-1] = -1.0/r[i]/dr + ivdr2
        A[i,i] = -2.0*ivdr2 - (att2)
        A[i,i+1] = 1.0/r[i]/dr + ivdr2
        F[i] = -(att2)*Pvv
    end

    # Boundary condtions for isolated tumor model
    A[1,1] = -2.0/dr^2 - (att^2)
    A[1,2] = 2.0/dr^2
    F[1] = -att2*Pvv
    A[N,N] = 1.0

    # Solution of linear problem for pressure distribution
    P = A\F
    return P
end

function Peclet(rs,N,R,Lpt,Svt,Kt,Pv,Pvv)
    P = Isolated_Pressure(N,R,Lpt,Svt,Kt,Pvv)
    Perm,sigma = solutePerm(Lpt,rs)
    return Pe_set = Lpt .* Pv .* (Pvv.*ones(N) - P) .* (1 - sigma) ./ Perm
end

function MST_Form(t, c, P, N, sigma, Perm, Lpt, Svt, Kt, Pvv, Pv, D, r, dr, kd)
    co = 1.0 # dimensionless drug concentration
    tspan = 300.0 # length of simulation type

    f = zeros(typeof(sigma), N)

    cv = co * exp(-t / kd)  # vascular concentration of the drug following exponential decay

    coeff1 = 2.0 * D / dr
    coeff2 = D / dr^2

    f[1] = 2.0*D*(c[2]-c[1])/dr^2 +
    Lpt*Svt*Pv*(Pvv-P[1])*(1-sigma)*cv

    #=
    Pe = Lpt * Pv * (Pvv - P[1]) * (1 - sigma) / Perm
    f[1] = 2 * D * (c[2] - c[1]) / dr^2 +
        Lpt * Svt * Pv * (Pvv - P[1]) * (1 - sigma) * cv +
        Perm * Svt * (cv - c[1]) * Pe / (exp(Pe) - 1.0)
    =#
    for j = 2:N-1
        f[j] = (coeff1/r[j])*(c[j+1]-c[j]) + coeff2*(c[j+1]-2.0*c[j]+c[j-1]) +
        Kt/(2*dr)*(P[j+1]-P[j-1])*((c[j+1]-c[j])/dr) +
        Lpt*Svt*Pv*(Pvv-P[j])*(1-sigma)*cv

        #=
        Pe = Lpt * Pv * (Pvv - P[j]) * (1 - sigma) / Perm
        f[j] = ((2 * D / r[j]) * ((c[j+1] - c[j]) / dr)) +
            (D * (c[j+1] - 2 * c[j] + c[j-1]) / dr^2) +
            Kt * ((P[j+1] - P[j-1]) / (2 * dr)) * ((c[j+1] - c[j]) / dr) +
            Lpt * Svt * Pv * (Pvv - P[j]) * (1 - sigma) * cv +
            Perm * Svt * (cv - c[j]) * Pe / (exp(Pe) - 1.0)
        =#
    end

    f[N] = 0.0
    return f
end

function EE_Form(x0,y0,h,n_out,i_out,P,N,sigma,Perm,Lpt,Svt,Kt,Pvv,Pv,D,r,dr,kd)
      xout = zeros(Float64,n_out+1)
      yout = zeros(typeof(sigma),n_out+1,length(y0))
      xout[1] = x0
      yout[1,:] = y0
      x = x0
      y = y0
      for j = 2:n_out+1
          for k = 1:i_out
              y = y + h*MST_Form(x,y,P,N,sigma,Perm,Lpt,Svt,Kt,Pvv,Pv,D,r,dr,kd)
              x = x + h
          end
          xout[j] = x
          yout[j,:] = y
      end
      return xout, yout
end


function Isolated_Model_Form(N,Kt,Lpt,Svt,R,Pv,Pvv,n_nodes,rs)

    #r = Vector(linspace(0,R,N))
    r = range(0,step = R/(N-1),length = N)
    r = r/R
    dr = 1.0/(N-1)

    # Solution of steady state pressure model
    P = Isolated_Pressure(N,R,Lpt,Svt,Kt,Pvv)
    Perm,sigma = solutePerm(Lpt,rs)
    D = Diffusion(rs)
    kd = Blood_half_life(rs)

    # Initial solute concentration
    c_0 = zeros(typeof(rs),N)

    t0 = 0.0
    tf = 300.0 # length of simulation (seconds)
    n_out = n_nodes - 1
    h = (tf - t0)/n_out;
    i_out = 1

    time, c = EE_Form(t0,c_0,h,n_out,i_out,P,N,sigma,Perm,Lpt,Svt,Kt,Pvv,Pv,D,r,dr,kd)
    return time, c
end

function solutePerm(Lpt,rs)
    # calculate diffusion coefficient from Stoke's Einstein
    kB = 1.380648*10.0^(-23)               # Boltzmann Constant (J/K)
    Temp = 310.15                           # Temperature K
    eta = 3*10.0^(-5)                      # viscosity of blood (mmHg-sec)
    conv_mu  = 133.322365                # (Pascal/mmHg)
    etac = eta*conv_mu                   # Pascal-sec
    pore_conv = 10.0^(-9)                  # (m/nm)
    r_partc = rs*pore_conv               # radius (m)
    D0 = kB*Temp/(6*pi*etac*r_partc)*1.0e4;   # Diffusivity (cm^2/s)

    # Bungay and Brenner
    a = [-73/60,77293/50400,-22.5083,-5.6117,-0.3363,-1.216,1.647]
    b = [7/60;-2227/50400;4.0180;-3.9788;-1.9215;4.392;5.006]

    # Calculate the pore size
    gamma = 1.0e-3
    eta = 3.0e-5                              # Blood viscosity (mmHg/s)
    L = 5.0e-4                                # Vessel wall thickness (cm)
    r_pore::typeof(rs) = sqrt(8*eta*L*Lpt/gamma)*1.0e7    # nm
    # rs = 30;                              # solute radius (nm) 30nm for FITC
    lambda = rs/r_pore
    t1 = zero(typeof(rs))
    t2 = zero(typeof(rs))
    p1 = zero(typeof(rs))
    p2 = zero(typeof(rs))
    for i = 1:7
        if i<3
            t1 = t1 + a[i]*(1-lambda)^i
            p1 = p1 + b[i]*(1-lambda)^i
        else
            t2 = t2 + a[i]*lambda^(i-3)
            p2 = p2 + b[i]*lambda^(i-3)
        end
    end
    Kt = t2 + 9.0/4.0*pi^2*sqrt(2.0)*(1.0+t1)*sqrt(1.0-lambda)^(-5)
    Ks = p2 + 9.0/4.0*pi^2*sqrt(2.0)*(1.0+p1)*sqrt(1.0-lambda)^(-5)
    Phi = (1.0-lambda)^2
    H = 6.0*pi*Phi/Kt
    W = Phi*(2.0-Phi)*Ks/(2.0*Kt)
    Perm = gamma*H*D0/L
    sigma = 1.0 - W
    return Perm,sigma
end

N = 100
n_nodes = 21

Svt = 200.0                # tumor vascular density
R = 1.0                    # tumor radius (cm)
Pv = 25.0                  # vascular pressure (mmHg)
Pvv = 1.0                  # vascu
PA=N,Svt,R,Pv,Pvv,n_nodes
function peak(dose,rs,param)
    N,Svt,R,Pv,Pvv,n_nodes=param
    Lpt = DEX_Lpt(dose)
    Kt = DEX_Kt(dose)
    tout,cout = Isolated_Model_Form(N,Kt,Lpt,Svt,R,Pv,Pvv,n_nodes,rs)
    cf = cout[convert(Int,round(n_nodes)),:]
    cpeak = cf[99]
end

cpeak(x)=peak(x[1],x[2],PA)
accum(x) =  output_scaler(cpeak(invscaler(x)))

accum (generic function with 1 method)

In [5]:
#Specify number of samples to generate and perform 70:30 split for data generation
n_samp=Int64(1e6);
n_train =Int64(n_samp*.7);
n_test = Int64(n_samp*.3);

In [6]:
#Create Data Sets
X=Float32.(reshape([x[j] for x in scaler.(sample(n_samp, lb,ub, SobolSample())) for j in eachindex(x)],(2,n_samp)));
Y=reshape([Float32.(y[j]) for y in [accum(col) for col in eachcol(X)] for j in eachindex(y)],(1,n_samp));

In [7]:
#Shuffle and Split data;
Xs,Ys=shuffleobs((X, Y));
(x_train, y_train), (x_test, y_test) = splitobs((Xs, Ys); at = 0.7);
#package data
using Flux.Data: DataLoader
bsize = n_train ÷ 10
data_mb = DataLoader((x_train,y_train),batchsize=bsize,shuffle=true);
data = DataLoader((x_train,y_train));

In [8]:
#Specify Neural Network Structure
n=8
activation_func=Flux.tanh
NN_peak = Chain(Dense(2,n),Dense(n,n,activation_func),Dense(n,n,activation_func),Dense(n,1))
loss(x,y) = Flux.mse(NN_peak(x),y)
learning_rate = 5e-4
optimizer = ADAM(learning_rate)
n_epochs = 50;
ps = Flux.params(NN_peak);
ϵ=1e-7;

In [9]:
Flux.train!(loss, ps, data, optimizer)
Flux.train!(loss, ps, data, optimizer)
@show loss(x_train,y_train)

for i=1:n_epochs
    Flux.train!(loss, ps, data_mb, optimizer)
    l=loss(x_train,y_train)
    #epoch_l[i]=l
    @show(i,l)
    if l < ϵ
        break
    end
end

loss(x_train, y_train) = 2.3826875f-5
i = 1
l = 2.2068332f-5
i = 2
l = 1.4095856f-5
i = 3
l = 1.3911018f-5
i = 4
l = 1.2469275f-5
i = 5
l = 1.2241704f-5
i = 6
l = 1.1928959f-5
i = 7
l = 1.1788524f-5
i = 8
l = 1.166305f-5
i = 9
l = 1.1575139f-5
i = 10
l = 1.1500364f-5
i = 11
l = 1.14392005f-5
i = 12
l = 1.1386275f-5
i = 13
l = 1.1340631f-5
i = 14
l = 1.1300081f-5
i = 15
l = 1.12640555f-5
i = 16
l = 1.123179f-5
i = 17
l = 1.1202313f-5
i = 18
l = 1.1175085f-5
i = 19
l = 1.11500985f-5
i = 20
l = 1.1126691f-5
i = 21
l = 1.1104702f-5
i = 22
l = 1.10839765f-5
i = 23
l = 1.1064292f-5
i = 24
l = 1.1045542f-5
i = 25
l = 1.10275305f-5
i = 26
l = 1.1010408f-5
i = 27
l = 1.0993796f-5
i = 28
l = 1.097772f-5
i = 29
l = 1.0962097f-5
i = 30
l = 1.0946884f-5
i = 31
l = 1.0932206f-5
i = 32
l = 1.0917803f-5
i = 33
l = 1.0903708f-5
i = 34
l = 1.0889958f-5
i = 35
l = 1.08764725f-5
i = 36
l = 1.0863164f-5
i = 37
l = 1.0850125f-5
i = 38
l = 1.083732f-5
i = 39
l = 1.0824681f-5
i = 40
l = 1.0812111f-5
i = 41
l 

In [10]:
@show loss(x_test,y_test)

loss(x_test, y_test) = 1.0643908f-5


1.0643908f-5

In [11]:
NN_model(x)=inv_output_scaler(NN_peak(vecscaler(x)))

NN_model (generic function with 1 method)

In [12]:
NN_model(lb)

1-element Vector{Float32}:
 1.180819

In [13]:
#Estimate Percent Error between ANN model and mechanistic model
#Must ignore first element because of divide by 0 error
y_test_inv_scaled= inv_output_scaler(y_test)
y_test_mod = inv_output_scaler(NN_peak(x_test))
percent_diff(orig,new) = orig == 0 ? NaN : abs((orig-new)/orig)*100
error_mat =percent_diff.(y_test_inv_scaled,y_test_mod)
mean_error = mean(error_mat);
std_error = std(error_mat);
@show mean_error;
@show std_error;

mean_error = 0.20307952f0
std_error = 0.24069569f0


In [14]:
function mean_error_func(x)
    orig = accumlation_intermediate(x)
    new = NN_model(x)[1]
    return percent_diff(orig,new)
end

mean_error_func (generic function with 1 method)

In [15]:
n_points = 100
dose_d=range(lb[1], ub[1], length=n_points)
rs_d= range(lb[2], ub[2], length=n_points)
err = reshape([mean_error_func([d,r]) for d in dose_d for r in rs_d],(n_points,n_points))

scale_factor =1
errplot = plot(dose_d,rs_d, err,title = "Approximate Percent Errort",
xlabel="Dose",
ylabel="Radius",
linetype=:heatmap,
size = (640*scale_factor,480*scale_factor),
dpi=200
);
display(errplot)

LoadError: UndefVarError: accumlation_intermediate not defined

In [17]:
using BSON: @save
@save "P:\\TherapyDrugDesign\\cpeakEE.bson" NN_peak